In [3]:
from scipy.linalg import eigh
from openfermion.transforms import get_sparse_operator
from openfermion.utils import pauli_exp_to_qasm
from openfermion.ops import QubitOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from pyscf import ci
from pyscf import mp
from openfermionpyscf import run_pyscf
from openfermion.hamiltonians import MolecularData


from numpy import array, concatenate, zeros
from numpy.random import randn
from scipy.optimize import minimize

from openfermion.config import *

from openfermion.hamiltonians import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.utils import uccsd_singlet_paramsize



In [4]:
#menentukan molekul yang akan ditinjau
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.7414]]]
basis = "sto-3g"
multiplicity = 1
charge = 0
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
h2_molecule

In [5]:
#informasi-informasi dari molekul
one_electron_integrals = h2_molecule.one_body_integrals
two_electron_integrals = h2_molecule.two_body_integrals
orbitals = h2_molecule.n_orbitals
qubit= h2_molecule.n_qubits
elektron=h2_molecule.n_electrons
overlap=h2_molecule.overlap_integrals
#asu=h2_molecule.ccsd_double_amps
#print(qubit)
#print("Energi Hartree=", h2_molecule.hf_energy)
#print("Energi FCI=", h2_molecule.fci_energy)
#print("Energi MP2=", h2_molecule.mp2_energy)
#print("koefisien h_pq=",one_electron_integrals)
#print("koefisien h_pqrs=",two_electron_integrals)
#print("orbitals =",orbitals)
#print("overlap=",overlap)
#print(elektron)

In [6]:
from openfermion.utils import jw_hartree_fock_state
referensi=jw_hartree_fock_state(elektron,orbitals)
print(referensi)

[0. 0. 0. 1.]


In [7]:
#from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
#print(h2_molecule.get_molecular_hamiltonian())
#fermion=get_fermion_operator(h2_molecule.get_molecular_hamiltonian())
#print(fermion)
#jw=jordan_wigner(fermion)
#print(jw)

In [8]:
print(uccsd_singlet_paramsize(h2_molecule.n_qubits,h2_molecule.n_electrons))

2


In [9]:
import numpy as np
from openfermion.utils import uccsd_singlet_generator, uccsd_generator
initial_amplitudes = np.linspace(0,0.05677,2)
uccsd_fermion=uccsd_singlet_generator(initial_amplitudes,qubit,elektron)
#b=uccsd_generator(initial_amplitudes,qubit,elektron)
print(uccsd_fermion)

-0.05677 [0^ 2 1^ 3] +
-0.05677 [1^ 3 0^ 2] +
0.05677 [2^ 0 3^ 1] +
0.05677 [3^ 1 2^ 0]


In [1]:
uccsd_jw=jordan_wigner(uccsd_fermion)
print(uccsd_jw)


NameError: name 'jordan_wigner' is not defined

In [11]:
uccsd_bk=bravyi_kitaev(uccsd_fermion)
print(uccsd_bk)

-0.0141925j [X0 Z1 Y2] +
-0.0141925j [X0 Z1 Y2 Z3] +
-0.0141925j [X0 Y2] +
-0.0141925j [X0 Y2 Z3] +
0.0141925j [Y0 Z1 X2] +
0.0141925j [Y0 Z1 X2 Z3] +
0.0141925j [Y0 X2] +
0.0141925j [Y0 X2 Z3]


In [13]:
from openfermion.ops import QubitOperator
from openfermion.utils import pauli_exp_to_qasm
for line in pauli_exp_to_qasm([QubitOperator('X0 Z1 Y2',-0.0141925),QubitOperator('X0 Z1 Y2 Z3',-0.0141925), QubitOperator('X0 Y2 ', -0.0141925), 
                              QubitOperator('X0 Y2 Z3',-0.0141925j),QubitOperator('Y0 Z1 X2', 0.0141925), QubitOperator('Y0 Z1 X2 Z3',0.0141925),
                              QubitOperator('Y0 X2',0.0141925), QubitOperator('Y0 X2 Z3',0.0141925)]):
    print(line)

H 0
Rx 1.5707963267948966 2
CNOT 0 1
CNOT 1 2
Rz -0.0141925 2
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 2
H 0
Rx 1.5707963267948966 2
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 2
H 0
Rx 1.5707963267948966 2
CNOT 0 2
Rz -0.0141925 2
CNOT 0 2
H 0
Rx -1.5707963267948966 2
H 0
Rx 1.5707963267948966 2
CNOT 0 2
CNOT 2 3
Rz -0.0 3
CNOT 2 3
CNOT 0 2
H 0
Rx -1.5707963267948966 2
Rx 1.5707963267948966 0
H 2
CNOT 0 1
CNOT 1 2
Rz 0.0141925 2
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 2
Rx 1.5707963267948966 0
H 2
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 2
Rx 1.5707963267948966 0
H 2
CNOT 0 2
Rz 0.0141925 2
CNOT 0 2
Rx -1.5707963267948966 0
H 2
Rx 1.5707963267948966 0
H 2
CNOT 0 2
CNOT 2 3
Rz 0.0141925 3
CNOT 2 3
CNOT 0 2
Rx -1.5707963267948966 0
H 2


In [95]:
#membuat ansatz

import openfermion as of
import cirq
import openfermioncirq as ofc
import numpy as np
class MyAnsatz(ofc.VariationalAnsatz):
    def params(self):
        '''Parameter of the ansatz'''
        return [cirq.Symbol('theta_0'),cirq.Symbol('theta_1')]
    def operations(self, qubits):
        '''Ansatz Prosedur'''
        q0, q1, q2, q3 = qubits
        yield cirq.H(q0),cirq.H(q1),cirq.H(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0),cirq.H(q1),cirq.H(q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q3)
        
        yield cirq.H(q0),cirq.H(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0),cirq.H(q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        yield cirq.H(q3)
        
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.H(q2),cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.H(q2),cirq.H(q3)
        
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q1),cirq.H(q2),cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q1),cirq.H(q2),cirq.H(q3)
        
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.H(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.H(q2)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q3)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.H(q3)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q1)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.H(q3)
        
        
        
        
        
        
    def _generate_qubits(set):
        return cirq.LineQubit.range(4)

In [96]:
ansatz=MyAnsatz()
print(ansatz.circuit)

0: ───H──────────@─────────────────────────────────────@───H──────────H───@───────────────────────────────@─────────H───H──────────@───────────────────────────────@──────────H──────────H──────────@──────────────────────────────────────@──────────H──────────X^(-3/7)───@───────────────────────────────@──────────X^(-3/7)───X^(-3/7)───@──────────────────────────────────────@──────────X^(-3/7)───X^(-3/7)───@──────────────────────────────────────@───X^(-3/7)───X^(-3/7)───@───────────────────────────────@──────────X^(-3/7)───
                 │                                     │                  │                               │                        │                               │                                │                                      │                                │                               │                                │                                      │                                │                                      │                         │    

In [97]:
import openfermionpyscf as ofpyscf
geometry = [('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.7414))]
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Perform electronic structure calculations and
# obtain Hamiltonian as an InteractionOperator
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)



In [108]:
# Define the objective function
objective = ofc.HamiltonianObjective(hamiltonian)

# Create preparation circuit
q0, q1, q2, q3 = ansatz.qubits
preparation_circuit = cirq.Circuit.from_ops(cirq.X(q3))

# Create variational study
study = ofc.VariationalStudy(
    name='my_hydrogen_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=preparation_circuit)

print(study.circuit)
print(study.num_params)
print(ansatz.default_initial_params())
print("The energy of the default initial guess is {}.".format(study.value_of(ansatz.default_initial_params())))

0: ───────H──────────@─────────────────────────────────────@───H──────────H───@───────────────────────────────@─────────H───H──────────@───────────────────────────────@──────────H──────────H──────────@──────────────────────────────────────@──────────H──────────X^(-3/7)───@───────────────────────────────@──────────X^(-3/7)───X^(-3/7)───@──────────────────────────────────────@──────────X^(-3/7)───X^(-3/7)───@──────────────────────────────────────@───X^(-3/7)───X^(-3/7)───@───────────────────────────────@──────────X^(-3/7)───
                     │                                     │                  │                               │                        │                               │                                │                                      │                                │                               │                                │                                      │                                │                                      │                      

In [109]:
# Choose an initial guess
#initial_guess =  np.linspace(0,0.05677,2)
#print('Value of initial guess: {}'.format(study.value_of(initial_guess)))

# Choose an algorithm
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'COBYLA'},
    options={'maxiter': 100})

# Create OptimizationParams object
optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=ansatz.default_initial_params())

# Run optimization
result = study.optimize(optimization_params)
print('Optimized value: {}'.format(result.optimal_value))
print("Optimized parameters:")
print(result.optimal_parameters)

Optimized value: -0.3706695638982433
Optimized parameters:
[-0.1839796   0.03046497]


In [102]:
print(np.linspace(0,0.05677,2))
print(molecule.ccsd_energy)

[0.      0.05677]
-1.1372703406409186


In [79]:
from openfermion.ops import QubitOperator
from openfermion.utils import pauli_exp_to_qasm
for line in pauli_exp_to_qasm([QubitOperator('X0 X1 X2 Y3',-0.0141925),QubitOperator('X0 X1 Y2 X3',-0.0141925), QubitOperator('X0 Y1 X2 X3', 0.0141925), 
                              QubitOperator('X0 Y1 Y2 Y3',-0.0141925j),QubitOperator('Y0 X1 X2 X3', 0.0141925), QubitOperator('Y0 X1 Y2 Y3',-0.0141925),
                              QubitOperator('Y0 Y1 X2 Y3',0.0141925), QubitOperator('Y0 Y1 Y2 X3',0.0141925)]):
    print(line)


H 0
H 1
H 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
H 2
Rx -1.5707963267948966 3
H 0
H 1
Rx 1.5707963267948966 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
Rx -1.5707963267948966 2
H 3
H 0
Rx 1.5707963267948966 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
H 2
H 3
H 0
Rx 1.5707963267948966 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
Rx -1.5707963267948966 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3
Rx 1.5707963267948966 0
H 1
H 2
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz 0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
H 2
H 3
Rx 1.5707963267948966 0
H 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.0141925 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
Rx -1.5707963267948966 0
H 1
Rx -1.5707963267948966 2
Rx -1.57079

In [107]:
from openfermion.utils import jw_configuration_state, jw_hartree_fock_state
print(molecule.n_orbitals)
print(molecule.n_electrons)
print(molecule.n_qubits)
referensi=jw_hartree_fock_state(2,2)
print(referensi)

2
2
4
[0. 0. 0. 1.]


In [23]:
print(fermion)
jw2=jordan_wigner(fermion)
print(jw2)

0.7151043390810812 [] +
-1.2533097866459773 [0^ 0] +
0.3373779634072241 [0^ 0^ 0 0] +
0.09060523100759854 [0^ 0^ 2 2] +
0.3373779634072241 [0^ 1^ 1 0] +
0.09060523100759854 [0^ 1^ 3 2] +
0.09060523100759854 [0^ 2^ 0 2] +
0.3318557006754069 [0^ 2^ 2 0] +
0.09060523100759854 [0^ 3^ 1 2] +
0.3318557006754069 [0^ 3^ 3 0] +
0.3373779634072241 [1^ 0^ 0 1] +
0.09060523100759854 [1^ 0^ 2 3] +
-1.2533097866459773 [1^ 1] +
0.3373779634072241 [1^ 1^ 1 1] +
0.09060523100759854 [1^ 1^ 3 3] +
0.09060523100759854 [1^ 2^ 0 3] +
0.3318557006754069 [1^ 2^ 2 1] +
0.09060523100759854 [1^ 3^ 1 3] +
0.3318557006754069 [1^ 3^ 3 1] +
0.3318557006754069 [2^ 0^ 0 2] +
0.09060523100759854 [2^ 0^ 2 0] +
0.3318557006754069 [2^ 1^ 1 2] +
0.09060523100759854 [2^ 1^ 3 0] +
-0.47506884877217576 [2^ 2] +
0.09060523100759854 [2^ 2^ 0 0] +
0.348825752245232 [2^ 2^ 2 2] +
0.09060523100759854 [2^ 3^ 1 0] +
0.348825752245232 [2^ 3^ 3 2] +
0.3318557006754069 [3^ 0^ 0 3] +
0.09060523100759854 [3^ 0^ 2 1] +
0.3318557006754069 

In [24]:
for line in pauli_exp_to_qasm([QubitOperator('X0 X1 Y2 Y3',-0.045302615503799264)]):
    print(line)

H 0
H 1
Rx 1.5707963267948966 2
Rx 1.5707963267948966 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
Rz -0.045302615503799264 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 1
Rx -1.5707963267948966 2
Rx -1.5707963267948966 3


In [20]:
from openfermion.ops import InteractionOperator
core_constat,  one_body_integrals, two_body_integrals = (molecule.get_active_space_integrals([0], [1]))
active_space_hamiltonian= InteractionOperator(core_constat,one_body_integrals,two_body_integrals)

In [21]:
print(active_space_hamiltonian)

() -1.8318636464775062
((0, 1), (0, 0)) 0.6711434919142547
((0, 1), (0, 1), (0, 0), (0, 0)) 0.6976515044904635

